In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_random_wikipedia_page=requests.get(wikipedia_link)
page=raw_random_wikipedia_page.text

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, 'html.parser')

In [4]:
x=[t.text for t in soup.find_all('td')]

In [5]:
rng=x.index("")
y=x[0:rng]

In [6]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
neighborhoods

,PostalCode,Borough,Neighborhood


In [8]:
for i in range(0,rng):
    if i%3==0:
        postal_code=y[i]
        borough=y[i+1]
        nbd=y[i+2]
#        print("postal_code:",postal_code)
#        print("borough:",borough)
#        print("neighborhood:",nbd)
        neighborhoods = neighborhoods.append({'PostalCode': postal_code,
                                              'Borough':  borough,
                                              'Neighborhood': nbd}, ignore_index=True)    

In [9]:
neighborhoods=neighborhoods.replace('\n','',regex=True)

In [10]:
neighborhoods=neighborhoods[neighborhoods.Borough != 'Not assigned']

In [11]:
#neighborhoods.groupby(neighborhoods["PostalCode"],neighborhoods["Borough"])['Neighborhood'].apply(lambda Neighborhood: ','.join(Neighborhood))
nb=neighborhoods.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda t: ','.join(t)).to_frame().reset_index()

In [12]:
nb.loc[nb['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood
57,M5G,Downtown Toronto,Central Bay Street


In [13]:
nb.loc[nb.Neighborhood == 'Not assigned', 'Neighborhood'] = nb.Borough

In [14]:
nb.shape
print('There are {} rows in the dataframe' .format(nb.shape[0]))

There are 103 rows in the dataframe


In [15]:
nb.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
import pandas as pd
file=pd.read_csv('http://cocl.us/Geospatial_data')

In [17]:
file.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
mrg=pd.concat([nb,file],axis=1)

In [19]:
mrg.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### Below is the 2nd dataframe created for the assignment

In [20]:
neighborhoods=mrg.drop(['Postal Code'],axis=1)
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


##### <b>Use geopy library to get the latitude and longitude values of Toronto</b>

In [21]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


##### <b>Create a map of Toronto with neighborhoods superimposed on top.</b>

In [22]:
import folium

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, pc in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood'],neighborhoods['PostalCode']):
    label = '{}, {}, {}'.format(neighborhood, borough, pc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
##### 

In [23]:
scarb_data = neighborhoods[neighborhoods['Borough'] == ('Scarborough')].reset_index(drop=True)
scarb_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [25]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [26]:
# create map of Scarborough using latitude and longitude values
map_scarb = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarb_data['Latitude'], scarb_data['Longitude'], scarb_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarb)  
    
map_scarb

In [27]:
CLIENT_ID = 'B3RG0FZMFK04VGBWNNBFSPU40XKJUN0GJL0ZA1MJY3ONRLYM' # your Foursquare ID
CLIENT_SECRET = 'Y1VLGS4MGY5MCDNJ3NXYGBKTNY3CSTLFOMTEWJCJO5FXCDCE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B3RG0FZMFK04VGBWNNBFSPU40XKJUN0GJL0ZA1MJY3ONRLYM
CLIENT_SECRET:Y1VLGS4MGY5MCDNJ3NXYGBKTNY3CSTLFOMTEWJCJO5FXCDCE


#### Let's create a function to repeat the same process to all the neighborhoods in Scarborough

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *scarb_venues*.

In [29]:
scarb_venues = getNearbyVenues(names=scarb_data['Neighborhood'],
                                   latitudes=scarb_data['Latitude'],
                                   longitudes=scarb_data['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West,Steeles West
Upper Rouge


In [30]:
scarb_venues.shape

(81, 7)

In [31]:
scarb_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Let's check how many venues were returned for each neighborhood

In [32]:
scarb_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea,Golden Mile,Oakridge",9,9,9,9,9,9
"Clarks Corners,Sullivan,Tam O'Shanter",9,9,9,9,9,9
"Cliffcrest,Cliffside,Scarborough Village West",2,2,2,2,2,2
"Dorset Park,Scarborough Town Centre,Wexford Heights",6,6,6,6,6,6
"East Birchmount Park,Ionview,Kennedy Park",7,7,7,7,7,7


#### Let's find out how many unique categories can be curated from all the returned venues

In [33]:
print('There are {} uniques categories.'.format(len(scarb_venues['Venue Category'].unique())))

There are 51 uniques categories.


## 3. Analyze Each Neighborhood

In [34]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarb_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
scarb_onehot.shape

(81, 52)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [59]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.5,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
2,"Birch Cliff,Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,...,0.000000,0.0,0.000000,0.000000,0.25,0.00,0.000000,0.000000,0.000000,0.000000
3,Cedarbrae,0.0,0.142857,0.142857,0.142857,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.142857,0.000000,0.000000
4,"Clairlea,Golden Mile,Oakridge",0.0,0.000000,0.222222,0.000000,0.0,0.000000,0.222222,0.111111,0.00,...,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.111111,0.000000,0.000000,0.000000
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.222222,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.111111,0.000000,0.000000
6,"Cliffcrest,Cliffside,Scarborough Village West",0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000
7,"Dorset Park,Scarborough Town Centre,Wexford He...",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.166667
8,"East Birchmount Park,Ionview,Kennedy Park",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.0,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.142857,0.000000
9,"Guildwood,Morningside,West Hill",0.0,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.00,...,0.166667,0.0,0.166667,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000


#### Let's print each neighborhood along with the top 5 most common venues

In [60]:
num_top_venues = 5

for hood in scarb_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarb_grouped[scarb_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0               Lounge  0.25
1       Breakfast Spot  0.25
2         Skating Rink  0.25
3       Clothing Store  0.25
4  American Restaurant  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2        American Restaurant   0.0
3          Korean Restaurant   0.0
4  Latin American Restaurant   0.0


----Birch Cliff,Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2                   Café  0.25
3  General Entertainment  0.25
4                   Park  0.00


----Cedarbrae----
                  venue  freq
0                Bakery  0.14
1                  Bank  0.14
2       Thai Restaurant  0.14
3    Athletics & Sports  0.14
4  Caribbean Restaurant  0.14


----Clairlea,Golden Mile,Oakridge----
          venue  freq
0        Bakery  0.22
1      Bus Line  0.22
2  In

###First let's write a function to sort the venues in descending order.

In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [62]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Coffee Shop
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Vietnamese Restaurant,Clothing Store,Grocery Store
2,"Birch Cliff,Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant
4,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant


In [63]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Coffee Shop
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Vietnamese Restaurant,Clothing Store,Grocery Store
2,"Birch Cliff,Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant
4,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant
5,"Clarks Corners,Sullivan,Tam O'Shanter",Pizza Place,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant
6,"Cliffcrest,Cliffside,Scarborough Village West",American Restaurant,Motel,Indian Restaurant,Hakka Restaurant,Grocery Store
7,"Dorset Park,Scarborough Town Centre,Wexford He...",Indian Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant
8,"East Birchmount Park,Ionview,Kennedy Park",Discount Store,Coffee Shop,Convenience Store,Hobby Shop,Department Store
9,"Guildwood,Morningside,West Hill",Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center


Run *k*-means to cluster the neighborhood into 5 clusters.

In [64]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 1, 1, 1, 1, 4, 1, 1, 1, 3, 1, 1, 2, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [65]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarb_merged = scarb_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarb_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3.0,Moving Target,Bar,Vietnamese Restaurant,Hakka Restaurant,Grocery Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,Grocery Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant


In [66]:
scarb_merged.fillna

<bound method DataFrame.fillna of    PostalCode      Borough                                       Neighborhood  \
0         M1B  Scarborough                                      Rouge,Malvern   
1         M1C  Scarborough               Highland Creek,Rouge Hill,Port Union   
2         M1E  Scarborough                    Guildwood,Morningside,West Hill   
3         M1G  Scarborough                                             Woburn   
4         M1H  Scarborough                                          Cedarbrae   
5         M1J  Scarborough                                Scarborough Village   
6         M1K  Scarborough          East Birchmount Park,Ionview,Kennedy Park   
7         M1L  Scarborough                      Clairlea,Golden Mile,Oakridge   
8         M1M  Scarborough      Cliffcrest,Cliffside,Scarborough Village West   
9         M1N  Scarborough                         Birch Cliff,Cliffside West   
10        M1P  Scarborough  Dorset Park,Scarborough Town Centre,Wexford He.

In [67]:
scarb_merged.fillna(0,inplace=True)

In [68]:
scarb_merged['Cluster Labels']

0     2.0
1     3.0
2     1.0
3     1.0
4     1.0
5     0.0
6     1.0
7     1.0
8     4.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    0.0
15    1.0
16    0.0
Name: Cluster Labels, dtype: float64

In [69]:
scarb_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3.0,Moving Target,Bar,Vietnamese Restaurant,Hakka Restaurant,Grocery Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,Grocery Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,1.0,Discount Store,Coffee Shop,Convenience Store,Hobby Shop,Department Store
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,1.0,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,4.0,American Restaurant,Motel,Indian Restaurant,Hakka Restaurant,Grocery Store
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant


In [70]:
#scarb_merged.head()
scarb_merged['Cluster Labels'] = scarb_merged['Cluster Labels'].astype(np.int64)
scarb_merged.dtypes
#scarb_merged['Cluster Labels']=scarb_merged['Cluster Labels'].astype(int)

PostalCode                object
Borough                   object
Neighborhood              object
Latitude                 float64
Longitude                float64
Cluster Labels             int64
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
dtype: object

Finally, let's visualize the resulting clusters

In [71]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighborhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [72]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Scarborough,0,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant
14,Scarborough,0,Playground,Park,Vietnamese Restaurant,Clothing Store,Grocery Store
16,Scarborough,0,0,0,0,0,0


#### Cluster 2

In [74]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Scarborough,1,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center
3,Scarborough,1,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,Grocery Store
4,Scarborough,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant
6,Scarborough,1,Discount Store,Coffee Shop,Convenience Store,Hobby Shop,Department Store
7,Scarborough,1,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant
9,Scarborough,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant
10,Scarborough,1,Indian Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,Vietnamese Restaurant
11,Scarborough,1,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant
12,Scarborough,1,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Coffee Shop
13,Scarborough,1,Pizza Place,Thai Restaurant,Fried Chicken Joint,Italian Restaurant,Fast Food Restaurant


#### Cluster 3

In [75]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Scarborough,2,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store


#### Cluster 4

In [76]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Scarborough,3,Moving Target,Bar,Vietnamese Restaurant,Hakka Restaurant,Grocery Store


#### Cluster 5

In [77]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Scarborough,4,American Restaurant,Motel,Indian Restaurant,Hakka Restaurant,Grocery Store
